# Topic Modeling using the BMW integrated report

In [14]:
# import the required packages

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'group', 'financial', 'report'])

In [16]:
# I previousely convert the pdf to txt and then to JSON using an online tool. 
import json

In [18]:
with open('bmw.json') as f:
    d = json.load(f)

{'BMW GROUP\nREPORT 2020\n\nOUR RESPONSIBILITY.\nOUR FUTURE.\nReport on the BMW Group’s economic\nperformance and its ecological and social\ncontributions.\n\nLINK TO THE ONLINE REPORT\n\n\x0cBMW Group\n\nContents\n\n2\n\nReport 2020\n\nCONTENTS\n\n35\n\n4\n\n7\n\nAbout This Report\n(Part of the Combined\nManagement Report)\n\nTO OUR\nSTAKEHOLDERS\n\n8\n\nBMW Group in Figures\n\n13\n\nReport of the Supervisory Board\n\n23\n\nStatement of the Chairman\nof the Board of Management\n\n29\n\nDialogue with Stakeholders\n\n33\n\nBMW Stock and Capital Markets\nin 2020\n\nCOMBINED\nMANAGEMENT\nREPORT\n\n36\n\nGeneral Information and Group Profile\n\n69\n\nProducts and Services\n\n87\n\nProduction, Purchasing and\nSupplier Network\n\n105\n\nEmployees and Society\n\n122\n\nReport on Economic Position\n\n159\n\nReport on Outlook, Risks and\nOpportunities\n\n179\n\nInternal Control System Relevant for\nAccounting and Financial Reporting\nProcesses\n\n180\n\nDisclosures Relevant for Takeovers\nand E

# Tokenize words and Clean-up text

simple_preprocess() makes most of the work

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(d))


[['bmw', 'group', 'report', 'our', 'responsibility', 'our', 'future', 'report', 'on', 'the', 'bmw', 'group', 'economic', 'performance', 'and', 'its', 'ecological', 'and', 'social', 'contributions', 'link', 'to', 'the', 'online', 'report', 'bmw', 'group', 'contents', 'report', 'contents', 'about', 'this', 'report', 'part', 'of', 'the', 'combined', 'management', 'report', 'to', 'our', 'stakeholders', 'bmw', 'group', 'in', 'figures', 'report', 'of', 'the', 'supervisory', 'board', 'statement', 'of', 'the', 'chairman', 'of', 'the', 'board', 'of', 'management', 'dialogue', 'with', 'stakeholders', 'bmw', 'stock', 'and', 'capital', 'markets', 'in', 'combined', 'management', 'report', 'general', 'information', 'and', 'group', 'profile', 'products', 'and', 'services', 'production', 'purchasing', 'and', 'supplier', 'network', 'employees', 'and', 'society', 'report', 'on', 'economic', 'position', 'report', 'on', 'outlook', 'risks', 'and', 'opportunities', 'internal', 'control', 'system', 'relevant

In [7]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


['bmw', 'group', 'report', 'our', 'responsibility', 'our', 'future', 'report', 'on', 'the', 'bmw', 'group', 'economic', 'performance', 'and', 'its', 'ecological', 'and', 'social', 'contributions', 'link', 'to', 'the', 'online', 'report', 'bmw', 'group', 'contents', 'report', 'contents', 'about', 'this', 'report', 'part', 'of', 'the', 'combined', 'management', 'report', 'to', 'our', 'stakeholders', 'bmw', 'group', 'in', 'figures', 'report', 'of', 'the', 'supervisory', 'board', 'statement', 'of', 'the', 'chairman', 'of', 'the', 'board', 'of', 'management', 'dialogue', 'with', 'stakeholders', 'bmw', 'stock', 'and', 'capital', 'markets', 'in', 'combined', 'management', 'report', 'general', 'information', 'and', 'group', 'profile', 'products', 'and', 'services', 'production', 'purchasing', 'and', 'supplier_network', 'employees', 'and', 'society', 'report', 'on', 'economic_position', 'report', 'on', 'outlook', 'risks', 'and', 'opportunities', 'internal_control_system', 'relevant', 'for', 'ac

In [8]:
import spacy

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram[bigram[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['responsibility', 'future', 'economic', 'performance', 'ecological', 'social', 'contribution', 'link', 'online', 'content', 'content', 'part', 'combined', 'management', 'stakeholder', 'figure', 'supervisory', 'board', 'statement', 'management', 'dialogue', 'stakeholder', 'stock', 'capital', 'market', 'combine', 'management', 'general', 'information', 'profile', 'product', 'service', 'production', 'purchase', 'supplier_network', 'employee', 'society', 'outlook', 'risk', 'opportunitie', 'internal_control', 'system', 'relevant', 'accounting', 'reporting', 'process', 'disclosure', 'relevant', 'takeover', 'explanatory_comment', 'content', 'statement', 'corporate_governance', 'fundamental_aspect', 'corporate_governance', 'part', 'combine', 'management', 'remuneration', 'part', 'combine', 'management', 'glossary', 'explanation', 'key', 'figure', 'income', 'statement', 'segment', 'statement', 'balance_sheet', 'segment', 'responsibility', 'statement', 'company', 'legal', 'representative', 'ca

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 5), (13, 1), (14, 1), (15, 1), (16, 5), (17, 1), (18, 1), (19, 1), (20, 1), (21, 6), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 4), (37, 1), (38, 3), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 3), (56, 1), (57, 1), (58, 1), (59, 1), (60, 8), (61, 3), (62, 1), (63, 4), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 12), (78, 3), (79, 2), (80, 1), (81, 1), (82, 1), (83, 4), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 2), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 4), (101, 3), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 2), (109, 2), (110, 1

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

the keywords for each topic and the weightage(importance) of each keyword is shown with lda_model.print_topics(). Output gives the top 10 keywords that contribute to this topic. The weights reflect how important a keyword is to that topic.

In [11]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.073*"board" + 0.034*"supervisory" + 0.033*"member" + 0.032*"remuneration" '
  '+ 0.023*"year" + 0.023*"management" + 0.015*"amount" + 0.012*"target" + '
  '0.011*"meeting" + 0.011*"share"'),
 (1,
  '0.016*"employee" + 0.014*"management" + 0.011*"also" + 0.009*"supplier" + '
  '0.009*"work" + 0.008*"sustainability" + 0.008*"health" + 0.008*"safety" + '
  '0.008*"measure" + 0.008*"gri"'),
 (2,
  '0.001*"year" + 0.001*"board" + 0.001*"risk" + 0.001*"management" + '
  '0.001*"amount" + 0.001*"service" + 0.001*"supervisory" + 0.001*"also" + '
  '0.001*"share" + 0.001*"statement"'),
 (3,
  '0.029*"share" + 0.024*"control" + 0.016*"right" + 0.015*"change" + '
  '0.014*"agreement" + 0.013*"board" + 0.013*"company" + 0.013*"article" + '
  '0.011*"management" + 0.009*"voting_right"'),
 (4,
  '0.017*"compliance" + 0.014*"management" + 0.012*"audit" + 0.010*"vehicle" + '
  '0.010*"statement" + 0.010*"risk" + 0.009*"also" + 0.009*"information" + '
  '0.008*"consolidated" + 0.008*"use"'),


In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.157199119434658

Coherence Score:  0.43395346425048276


In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/Felipe/Desktop/LDA_BMW/env/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.177872  0.086056       1        1  19.106432
10     0.107743 -0.001481       2        1  16.171492
19     0.058550 -0.119397       3        1  16.146724
4      0.035347 -0.064397       4        1   9.338394
0      0.185184  0.101746       5        1   8.697605
11     0.018708 -0.172759       6        1   6.473217
7      0.089273 -0.094251       7        1   6.000586
1      0.015822 -0.138217       8        1   4.642258
5      0.010454 -0.153040       9        1   4.059627
16     0.135722  0.109397      10        1   2.937692
13     0.071512  0.155572      11        1   1.918601
12    -0.038910 -0.061494      12        1   1.644532
3     -0.004839  0.147234      13        1   1.417602
18    -0.049920 -0.104889      14        1   0.980923
17    -0.091243  0.046986      15        1   0.375202
15    -0.147576  0.039812      16        1   0.035413
8     -0.112023  0.056526      17        1   0.025727
14    -0.139957  0.052369      18        1   0.014477
6     -0.169460  0.055957      19        1   0.009481
2     -0.152261  0.058270      20        1   0.004015, topic_info=             Term        Freq       Total Category  logprob  loglift
320  remuneration  323.000000  323.000000  Default  30.0000  30.0000
42          board  566.000000  566.000000  Default  29.0000  29.0000
361     statement  460.000000  460.000000  Default  28.0000  28.0000
413          year  752.000000  752.000000  Default  27.0000  27.0000
334          risk  624.000000  624.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
874        member    0.001130  247.637237  Topic20  -7.6880  -2.1743
320  remuneration    0.001169  323.125213  Topic20  -7.6545  -2.4070
54         change    0.001090  235.256413  Topic20  -7.7246  -2.1597
36           base    0.001106  276.273787  Topic20  -7.7102  -2.3060
63        company    0.001094  255.866535  Topic20  -7.7207  -2.2398

[1446 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
4185      4  0.869971        abs_hgb
4025     10  0.964602  absolute_term
1         1  0.180243     accessible
1         3  0.180243     accessible
1         6  0.180243     accessible
...     ...       ...            ...
413      12  0.009305           year
413      14  0.002659           year
413      15  0.001329           year
2357      3  0.933286          young
4076     10  0.936330        zufluss

[3635 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 11, 20, 5, 1, 12, 8, 2, 6, 17, 14, 13, 4, 19, 18, 16, 9, 15, 7, 3])